In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoarthritis"
cohort = "GSE142049"

# Input paths
in_trait_dir = "../../input/GEO/Osteoarthritis"
in_cohort_dir = "../../input/GEO/Osteoarthritis/GSE142049"

# Output paths
out_data_file = "../../output/preprocess/Osteoarthritis/GSE142049.csv"
out_gene_data_file = "../../output/preprocess/Osteoarthritis/gene_data/GSE142049.csv"
out_clinical_data_file = "../../output/preprocess/Osteoarthritis/clinical_data/GSE142049.csv"
json_path = "../../output/preprocess/Osteoarthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional data of inflamatory arthritis B cells"
!Series_summary	"With a focus on rheumatoid arthritis (RA), we sought new insight into genetic mechanisms of adaptive immune dysregulation to help prioritise molecular pathways for targeting in this and related immune pathologies. Whole genome methylation and transcriptional data from isolated CD4+ T cells and B cells of >100 genotyped and phenotyped inflammatory arthritis patients, all of whom were naïve to immunomodulatory treatments, were obtained. Analysis integrated these comprehensive data with GWAS findings across IMDs and other publically available resources."
!Series_overall_design	"Suspected inflammatory arthritis patients of Northern European ancestry were recruited prior to treatment with immunomodulatory drugs. RA patients were classified using current, internationally accepted criteria, and matched with disease controls in respect of demographic and clinical characteristics. CD19

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains transcriptional data
# from CD19+ B cells, which suggests gene expression data is available
is_gene_available = True

# 2.1 Data Availability
# Trait data: Look for Osteoarthritis in the sample characteristics
# Keys 5 and 6 contain diagnosis information, with key 6 having "working_diagnosis: Osteoarthritis"
trait_row = 6  # working_diagnosis field contains Osteoarthritis

# Age data: Available in key 2
age_row = 2  # age field

# Gender data: Available in key 1
gender_row = 1  # gender field

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert osteoarthritis diagnosis to binary (0/1)"""
    if value is None or ':' not in value:
        return None
    
    diagnosis = value.split(':', 1)[1].strip()
    # Return 1 if diagnosis is Osteoarthritis, 0 otherwise
    if diagnosis == 'Osteoarthritis':
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age string to numeric value"""
    if value is None or ':' not in value:
        return None
    
    age_str = value.split(':', 1)[1].strip()
    try:
        return int(age_str)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if value is None or ':' not in value:
        return None
    
    gender = value.split(':', 1)[1].strip()
    if gender.upper() == 'F':
        return 0
    elif gender.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata - Initial filtering
# Trait data availability is determined by whether trait_row is None
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Create directory for clinical data if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Load the clinical data (assuming it was previously loaded to clinical_data)
    # Extract clinical features using the geo_select_clinical_features function
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    print("Preview of clinical data:")
    print(preview_df(clinical_df))
    
    # Save clinical data to CSV
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM4218229': [0.0, 82.0, 1.0], 'GSM4218230': [0.0, 29.0, 1.0], 'GSM4218231': [0.0, 61.0, 0.0], 'GSM4218232': [0.0, 56.0, 1.0], 'GSM4218233': [0.0, 55.0, 0.0], 'GSM4218234': [0.0, 72.0, 1.0], 'GSM4218235': [0.0, 50.0, 0.0], 'GSM4218236': [0.0, 22.0, 1.0], 'GSM4218237': [0.0, 61.0, 1.0], 'GSM4218238': [0.0, 53.0, 0.0], 'GSM4218239': [0.0, 54.0, 0.0], 'GSM4218240': [0.0, 20.0, 0.0], 'GSM4218241': [0.0, 49.0, 0.0], 'GSM4218242': [0.0, 61.0, 1.0], 'GSM4218243': [1.0, 53.0, 0.0], 'GSM4218244': [0.0, 59.0, 1.0], 'GSM4218245': [0.0, 56.0, 0.0], 'GSM4218246': [0.0, 57.0, 1.0], 'GSM4218247': [0.0, 55.0, 1.0], 'GSM4218248': [0.0, 56.0, 0.0], 'GSM4218249': [0.0, 35.0, 0.0], 'GSM4218250': [0.0, 58.0, 0.0], 'GSM4218251': [0.0, 69.0, 1.0], 'GSM4218252': [0.0, 46.0, 0.0], 'GSM4218253': [1.0, 57.0, 0.0], 'GSM4218254': [0.0, 50.0, 0.0], 'GSM4218255': [0.0, 66.0, 0.0], 'GSM4218256': [0.0, 52.0, 0.0], 'GSM4218257': [0.0, 63.0, 0.0], 'GSM4218258': [0.0, 51.0, 1.0], 'GSM4218259':

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Osteoarthritis/GSE142049/GSE142049_series_matrix.txt.gz
Gene data shape: (11809, 114)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651262', 'ILMN_1651278',
       'ILMN_1651315', 'ILMN_1651316', 'ILMN_1651341', 'ILMN_1651343',
       'ILMN_1651346', 'ILMN_1651347', 'ILMN_1651378', 'ILMN_1651385',
       'ILMN_1651405', 'ILMN_1651415', 'ILMN_1651429', 'ILMN_1651433'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# These identifiers start with "ILMN_" which indicates they are Illumina probe IDs
# from a microarray platform, not standard human gene symbols
# Illumina probe IDs need to be mapped to human gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Probe_Id', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], '

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examine the gene identifiers from the expression data and the annotation data
# From previous output, we can see:
# - Gene expression data index contains Illumina probe IDs like 'ILMN_1343291'
# - Gene annotation dataframe has 'ID' column that contains the same format of identifiers
# - 'Symbol' column in gene annotation contains gene symbols

# 2. Extract gene mapping using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
print(f"Gene mapping shape: {gene_mapping.shape}")
print("Preview of gene mapping:")
print(preview_df(gene_mapping, n=5))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This handles many-to-many relationships by dividing expression values for probes 
# that map to multiple genes, then summing values for each gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# Create directory for gene data if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save gene expression data to CSV file
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping shape: (44837, 2)
Preview of gene mapping:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}
Gene expression data shape after mapping: (8480, 114)
First 10 gene symbols after mapping:
['A2LD1', 'AAAS', 'AADACL1', 'AAGAB', 'AAMP', 'AARS', 'AARS2', 'AARSD1', 'AASDH', 'AASDHPPT']


Gene expression data saved to ../../output/preprocess/Osteoarthritis/gene_data/GSE142049.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data 
# (This was already done in the previous step, so no need to repeat)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Examine the sample IDs in the gene expression data to understand the structure
print("Gene data column names (sample IDs):")
print(gene_data.columns[:5])  # Print first 5 for brevity

# Inspect the clinical data format from the matrix file directly
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
print("\nRaw clinical data structure:")
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data columns: {clinical_data.columns[:5]}")

# Get the sample characteristics to re-extract the disease information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# 3. Directly create clinical features from the raw data again
# Verify trait row contains the disease information (OA vs RA)
print("\nValues in trait row:")
trait_values = clinical_data.iloc[trait_row].values
print(trait_values[:5])

# Create clinical dataframe with proper structure
# First get the sample IDs from gene data as these are our actual sample identifiers
sample_ids = gene_data.columns.tolist()

# Create the clinical features dataframe with those sample IDs
clinical_features = pd.DataFrame(index=[trait], columns=sample_ids)

# Fill the clinical features with our trait values by mapping GSM IDs to actual values
for col in clinical_data.columns:
    if col in sample_ids:
        # Extract the disease value and convert it
        disease_val = clinical_data.iloc[trait_row][col]
        clinical_features.loc[trait, col] = convert_trait(disease_val)

print("\nCreated clinical features dataframe:")
print(f"Shape: {clinical_features.shape}")
print(clinical_features.iloc[:, :5])  # Show first 5 columns

# 4. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"\nLinked data shape before handling missing values: {linked_data.shape}")

# 5. Handle missing values - we need to use the actual column name, not the trait variable
# First identify the actual trait column name in the linked data
trait_column = clinical_features.index[0]  # This should be 'Osteoarthritis'
print(f"Actual trait column in linked data: {trait_column}")

# Now handle missing values with the correct column name
linked_data_clean = handle_missing_values(linked_data, trait_column)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 6. Evaluate bias in trait and demographic features
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait_column)

# 7. Conduct final quality validation
note = "Dataset contains gene expression data from synovial fibroblasts of RA and OA patients. Data includes high serum and low serum responses."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=(linked_data_clean.shape[0] > 0),
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 8. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Normalized gene data shape: (8480, 114)
Gene data column names (sample IDs):
Index(['GSM4218229', 'GSM4218230', 'GSM4218231', 'GSM4218232', 'GSM4218233'], dtype='object')

Raw clinical data structure:
Clinical data shape: (7, 115)
Clinical data columns: Index(['!Sample_geo_accession', 'GSM4218229', 'GSM4218230', 'GSM4218231',
       'GSM4218232'],
      dtype='object')

Sample characteristics dictionary:
{0: ['patient: 2367', 'patient: 2390', 'patient: 2368', 'patient: 2437', 'patient: 2439', 'patient: 2379', 'patient: 1010', 'patient: 841', 'patient: 926', 'patient: 948', 'patient: 898', 'patient: 980', 'patient: 2145', 'patient: 2036', 'patient: 2040', 'patient: 2029', 'patient: 2044', 'patient: 2030', 'patient: 2045', 'patient: 2231', 'patient: 2257', 'patient: 2233', 'patient: 2261', 'patient: 1070', 'patient: 1054', 'patient: 1072', 'patient: 1076', 'patient: 1080', 'patient: 1083', 'patient: 2047'], 1: ['gender: M', 'gender: F'], 2: ['age: 82', 'age: 29', 'age: 61', 'age: 56', 'a

/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Linked data shape after handling missing values: (114, 8481)
For the feature 'Osteoarthritis', the least common label is '1' with 6 occurrences. This represents 5.26% of the dataset.
The distribution of the feature 'Osteoarthritis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Osteoarthritis/GSE142049.csv
